In [1]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import concatenate, BatchNormalization, Dropout
from tensorflow.keras import Model


/home/sebasmos/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sebasmos/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sebasmos/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sebasmos/anaconda3/envs/solaris

In [2]:

def cosmiq_sn4_baseline(input_shape=(512, 512, 3), base_depth=64):
    """Keras implementation of untrained TernausNet model architecture.

    Arguments:
    ----------
    input_shape (3-tuple): a tuple defining the shape of the input image.
    base_depth (int): the base convolution filter depth for the first layer
        of the model. Must be divisible by two, as the final layer uses
        base_depth/2 filters. The default value, 64, corresponds to the
        original TernausNetV1 depth.

    Returns:
    --------
    An uncompiled Keras Model instance with TernausNetV1 architecture.

    """
    inputs = Input(input_shape)
    conv1 = Conv2D(base_depth, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2_1 = Conv2D(base_depth*2, 3, activation='relu',
                     padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2_1)

    conv3_1 = Conv2D(base_depth*4, 3, activation='relu',
                     padding='same')(pool2)
    conv3_2 = Conv2D(base_depth*4, 3, activation='relu',
                     padding='same')(conv3_1)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3_2)

    conv4_1 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(pool3)
    conv4_2 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(conv4_1)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4_2)

    conv5_1 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(pool4)
    conv5_2 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(conv5_1)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5_2)

    conv6_1 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(pool5)

    up7 = Conv2DTranspose(base_depth*4, 2, strides=(2, 2), activation='relu',
                          padding='same')(conv6_1)
    concat7 = concatenate([up7, conv5_2])
    conv7_1 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(concat7)

    up8 = Conv2DTranspose(base_depth*4, 2, strides=(2, 2), activation='relu',
                          padding='same')(conv7_1)
    concat8 = concatenate([up8, conv4_2])
    conv8_1 = Conv2D(base_depth*8, 3, activation='relu',
                     padding='same')(concat8)

    up9 = Conv2DTranspose(base_depth*2, 2, strides=(2, 2), activation='relu',
                          padding='same')(conv8_1)
    concat9 = concatenate([up9, conv3_2])
    conv9_1 = Conv2D(base_depth*4, 3, activation='relu',
                     padding='same')(concat9)

    up10 = Conv2DTranspose(base_depth, 2, strides=(2, 2), activation='relu',
                           padding='same')(conv9_1)
    concat10 = concatenate([up10, conv2_1])
    conv10_1 = Conv2D(base_depth*2, 3, activation='relu',
                      padding='same')(concat10)

    up11 = Conv2DTranspose(int(base_depth/2), 2, strides=(2, 2),
                           activation='relu', padding='same')(conv10_1)
    concat11 = concatenate([up11, conv1])

    out = Conv2D(1, 1, activation='sigmoid', padding='same')(concat11)

    return Model(inputs=inputs, outputs=out)

In [3]:
custom_model_dict = {'model_name': 'xdxd_spacenet4_custom',
                     'weight_path': None,
                     'weight_url': None,
                     'arch': cosmiq_sn4_baseline}

In [4]:
import solaris as sol

In [5]:
path_to_model = "/home/sebasmos/Documentos/TRDP/DATASET/API/models"

model_name = '/xdxd_spacenet4_custom.yml'

fullpath = "%s%s" % ( path_to_model, model_name ) # for Linux

fullpath

'/home/sebasmos/Documentos/TRDP/DATASET/API/models/xdxd_spacenet4_custom.yml'

In [6]:
config = sol.utils.config.parse(fullpath)
config


{'model_name': 'xdxd_spacenet4_custom',
 'model_path': None,
 'train': True,
 'infer': True,
 'pretrained': False,
 'nn_framework': 'torch',
 'batch_size': 12,
 'data_specs': {'width': 512,
  'height': 512,
  'dtype': None,
  'image_type': 'zscore',
  'rescale': False,
  'rescale_minima': 'auto',
  'rescale_maxima': 'auto',
  'channels': 4,
  'label_type': 'mask',
  'is_categorical': False,
  'num_classes': 1,
  'mask_channels': 1,
  'val_holdout_frac': 0.2,
  'data_workers': None},
 'training_data_csv': '/home/sebasmos/Documentos/TRDP/DATASET/SN7_buildings_train/csvs/sn7_baseline_train_df.csv',
 'validation_data_csv': None,
 'inference_data_csv': '/home/sebasmos/Documentos/TRDP/DATASET/SN7_buildings_train/csvs/sn7_baseline_test_public_df.csv',
 'training_augmentation': {'augmentations': {'DropChannel': {'idx': 3,
    'axis': 2},
   'HorizontalFlip': {'p': 0.5},
   'RandomRotate90': {'p': 0.5},
   'RandomCrop': {'height': 512, 'width': 512, 'p': 1.0},
   'Normalize': {'mean': [0.006479

In [8]:
trainer = sol.nets.train.Trainer(config, custom_model_dict=custom_model_dict)

TypeError: cosmiq_sn4_baseline() got an unexpected keyword argument 'pretrained'